This notebook was originally part of `galaxy properties by environment-updated.ipynb` and studies what is going on with the bump that we observe for faint objects in the $V^2$ wall distribution.

In [1]:
import numpy as np

from astropy.table import Table
from astropy.io import fits

from sklearn.neighbors import NearestNeighbors

import matplotlib
import matplotlib.pyplot as plt
from matplotlib import ticker
%matplotlib notebook

# Data

In [2]:
#data_filename = 'data_flags_updated_Nov1.dat'
data_filename = '../../../../data/NSA_v1_0_1_VAGC_vflag-V2-VF.fits'

#catalog_SDSS = Table.read(data_filename, format='ascii')
hdu = fits.open(data_filename)
data = Table(hdu[1].data)
hdu.close()

## Keep the main SDSS footprint

In [3]:
# When the catalog is the NSA, only keep those objects which were originally in the DR7 LSS
catalog_SDSS = data[data['IN_DR7_LSS'] == 1]

ra_boolean = np.logical_and(catalog_SDSS['RA'] > 110, catalog_SDSS['RA'] < 270)
catalog_north = catalog_SDSS[ra_boolean]

strip_boolean = np.logical_and.reduce([catalog_north['RA'] > 250, 
                                       catalog_north['RA'] < 269, 
                                       catalog_north['DEC'] > 51, 
                                       catalog_north['DEC'] < 67])
catalog_main = catalog_north[~strip_boolean]

# What is up with the bump that we see in the $M_*$ distribution?

In [ ]:
all_survey = data[data['redshift'] < 0.1026]  # galaxies within the redshift cut

glx_excess= np.logical_and(all_survey['rabsmag']>-18.2,all_survey['rabsmag']<-16.8)

wall_vf_excess = np.logical_and(np.logical_and(all_survey['rabsmag']>-18.2,all_survey['rabsmag']<-16.8), all_survey['flag_voidfinder'] == 0)
# void_vf_excess = np.logical_and(np.logical_and(all_survey['rabsmag']>-18.2,all_survey['rabsmag']<-16.8), all_survey['flag_voidfinder'] == 1)
wall_v2_range = np.logical_and(np.logical_and(all_survey['rabsmag']>b,all_survey['rabsmag']<a), all_survey['flag_vsquared'] == 0)
    
wall_v2_excess = np.logical_and(np.logical_and(all_survey['rabsmag']>-18.2,all_survey['rabsmag']<-16.8), all_survey['flag_vsquared'] == 0)
void_v2_excess = np.logical_and(np.logical_and(all_survey['rabsmag']>-18.2,all_survey['rabsmag']<-16.8), all_survey['flag_vsquared'] == 1)
    
wall_v2_bool= (all_survey['flag_vsquared'] == 0)

##### Nearest Neighbours

In [ ]:
# wall_v2_range = np.logical_and(np.logical_and(all_survey['rabsmag']>b,all_survey['rabsmag']<a), all_survey['flag_vsquared'] == 0)
#     void_v2_rand = np.logical_and(np.logical_and(all_survey['rabsmag']>b,all_survey['rabsmag']<a), all_survey['flag_vsquared'] == 1)
galaxy_table_wall_v2=all_survey[wall_v2_bool]
galaxy_table_wall_v2_xyz=ra_dec_z_to_xyz(galaxy_table_wall_v2)

X1 = galaxy_table_wall_v2_xyz
Y1= galaxy_table_wall_v2_xyz
nbrs = NearestNeighbors(n_neighbors=2, algorithm='ball_tree').fit(X1)
distances, indices = nbrs.kneighbors(Y1)
distances_wall_v2=distances[:,1]

In [ ]:
plt.figure(figsize=(12,7))

plt.scatter(all_survey[wall_v2_bool]['rabsmag'],distances_wall_v2,alpha=0.1, s=1, color="g") # void vf
plt.ylim(0,12)
plt.xlim(-10,-24)
plt.xlabel("absolute magnitude", fontsize=14)
plt.title("WALL V2")
plt.ylabel("distance to nearest neighbor", fontsize=14);
# the deeper objects are brighter

In [ ]:
# ## x y z radius flag r ra dec
# maximal_=np.loadtxt('ALL_comoving_maximal.txt')

In [ ]:
# ra_maximal=maximal_[:,6]
# dec_maximal=maximal_[:,7]
# r_maximal=maximal_[:,5]

In [ ]:
# c = 299792
# def f(a,omega_m):
#      return 1/(np.sqrt(a*omega_m*(1+((1-omega_m)*a**3/omega_m))))

# def Distance(z,omega_m,h):
#     dist = np.ones(len(z))
#     H0 = 100*h
#     for i,redshift in enumerate(z):
#         a_start = 1/(1+redshift)
#         I = quad(f,a_start,1,args=omega_m)
#         dist[i] = I[0]*(c/H0)
#     return dist

In [ ]:
a_=[]
b_=[]
for i in [-13,-14,-15,-16,-17,-18,-19,-20,-21,-22]:
    a = i#np.random.uniform(-22+(18.2-16.8),-16+(18.2-16.8))
    b = a-(18.2-16.8)
    a_.append(a)
    b_.append(b)
    
#     wall_vf_rand = np.logical_and(np.logical_and(all_survey['rabsmag']>b,all_survey['rabsmag']<a), all_survey['flag_voidfinder'] == 0)
#     void_vf_rand = np.logical_and(np.logical_and(all_survey['rabsmag']>b,all_survey['rabsmag']<a), all_survey['flag_voidfinder'] == 1)
def range_neig(a,b):
    wall_v2_range = np.logical_and(np.logical_and(all_survey['rabsmag']>b,all_survey['rabsmag']<a), all_survey['flag_vsquared'] == 0)
#     void_v2_rand = np.logical_and(np.logical_and(all_survey['rabsmag']>b,all_survey['rabsmag']<a), all_survey['flag_vsquared'] == 1)
    galaxy_table_range_wall_v2=all_survey[wall_v2_range]
    galaxy_table_range_wall_v2_xyz=ra_dec_z_to_xyz(galaxy_table_range_wall_v2)
    
    X1 = galaxy_table_wall_v2_xyz # all wall galaxies
    Y1= galaxy_table_range_wall_v2_xyz 
    nbrs = NearestNeighbors(n_neighbors=2, algorithm='ball_tree').fit(X1)
    distances, indices = nbrs.kneighbors(Y1)
    distances_range=distances[:,1]
    return distances_range

plt.figure(figsize=(20,14))
for i in range(len(a_)):
    plt.hist(range_neig(a_[i],b_[i]), bins=50, density=True, label=str((a_[i],b_[i])), histtype='step')
# plt.hist(range_neig(a_[-1],b_[-1]), bins=200, density=True, label=str((a,b)), histtype='step')
# plt.hist(distances_range, bins=500, density=True, label=str((a,b)), histtype='step')
# plt.hist(distances_range, bins=500, density=True, label=str((a,b)), histtype='step')
# plt.hist(distances_range, bins=500, density=True, label=str((a,b)), histtype='step')
# plt.hist(distances_range, bins=500, density=True, label=str((a,b)), histtype='step')
# plt.hist(distances_range, bins=500, density=True, label=str((a,b)), histtype='step')
# plt.hist(distances_range, bins=500, density=True, label=str((a,b)), histtype='step')
# plt.hist(distances_range, bins=500, density=True, label=str((a,b)), histtype='step')
#     plt.hist(distances_excess, bins=100,density=True, alpha=0.5, label="excess")
plt.xlim(0,4)
plt.legend();

#     plt.scatter(np.array(all_survey[void_v2_rand]['ra']),np.array(all_survey[void_v2_rand]['dec']))
#     plt.title(str((a,b))+"  V2 void" )
#     plt.xlabel("ra")
#     plt.ylabel("dec")
# #     plt.xlim(-10,370)
# #     plt.ylim(-30,85)
#     plt.show()
#     plt.hist(np.array(all_survey[void_v2_rand]['redshift']))
# #     plt.xlim(0,0.1026)
#     plt.xlabel("redshift"+str((a,b)))
#     plt.ylabel("count");

In [ ]:
# ind_gal_depth_voidv2_new = np.load('galaxy_in_maximal_indices', allow_pickle=True)

## Bump depths

In [ ]:
ind_gal_depth_voidv2_new = np.load('ind_gal_depth_voidv2_equal_slices.npy', allow_pickle=True)

In [ ]:
np.sum(wall_v2_bool), np.sum(wall_v2_excess)

### Save excess range for visualization

In [ ]:
ra_excess=np.array(all_survey[wall_v2_bool]['ra'])
redshift_excess=np.array(all_survey[wall_v2_bool]['redshift'])
dec_excess=np.array(all_survey[wall_v2_bool]['dec'])
Rgal_excess=np.array(all_survey[wall_v2_bool]['Rgal'])
rabsmag_excess=np.array(all_survey[wall_v2_bool]['rabsmag'])

In [ ]:
galaxies_excess_table = Table([ra_excess,dec_excess,redshift_excess,Rgal_excess,rabsmag_excess], 
                              names=('ra','dec','redshift','Rgal','rabsmag'))
galaxies_excess_table.write('wall_v2_bool.dat', format='ascii.commented_header', overwrite=True)  

### Nearest distance of excess range

In [ ]:
galaxy_table_excess_wall_v2=all_survey[wall_v2_excess]
galaxy_table_wall_v2=all_survey[wall_v2_bool]
galaxy_table_range_wall_v2=all_survey[wall_v2_range]

In [ ]:
def ra_dec_z_to_xyz(galaxy_table):
    h=1
    DtoR = np.pi/180.
    RtoD = 180./np.pi
    c = 3e5

    r_gal = c*galaxy_table['redshift'].data/(100*h)
    ra = galaxy_table['ra'].data
    dec = galaxy_table['dec'].data
    
    ra_radian = ra*DtoR

    dec_radian = dec*DtoR

    x_gal = r_gal*np.cos(ra_radian)*np.cos(dec_radian)

    y_gal = r_gal*np.sin(ra_radian)*np.cos(dec_radian)

    z_gal = r_gal*np.sin(dec_radian)

    num_gal = x_gal.shape[0]

    gals_xyz = np.concatenate((x_gal.reshape(num_gal,1),
                               y_gal.reshape(num_gal,1),
                               z_gal.reshape(num_gal,1)), 
                              axis=1)
    return gals_xyz

In [ ]:
galaxy_table_excess_wall_v2_xyz=ra_dec_z_to_xyz(galaxy_table_excess_wall_v2)
galaxy_table_wall_v2_xyz=ra_dec_z_to_xyz(galaxy_table_wall_v2)
galaxy_table_range_wall_v2_xyz=ra_dec_z_to_xyz(galaxy_table_range_wall_v2)

In [ ]:
# tree = neighbors.KDTree(gals_xyz, metric = 'euclidean')
# ind =  tree.query_radius(gals_xyz_all, r =i*radius_voidv2)

In [ ]:
X1 = galaxy_table_wall_v2_xyz # all wall galaxies
Y1= galaxy_table_range_wall_v2_xyz 
nbrs = NearestNeighbors(n_neighbors=2, algorithm='ball_tree').fit(X1)
distances, indices = nbrs.kneighbors(Y1)
distances_range=distances[:,1]

In [ ]:
X = galaxy_table_wall_v2_xyz # all wall galaxies
Y= galaxy_table_excess_wall_v2_xyz # excess galaxies
nbrs = NearestNeighbors(n_neighbors=2, algorithm='ball_tree').fit(X)
distances, indices = nbrs.kneighbors(Y)
distances_excess=distances[:,1]

In [ ]:
plt.figure(figsize=(15,8))
plt.hist(distances_range, bins=1000, density=True, label="random")
plt.hist(distances_excess, bins=100,density=True, alpha=0.5, label="excess")
plt.legend();

In [ ]:
# this finds the the closest neighbours from X to Y and their distances.
X = [[0,0],[1,1]]
Y=[[0,2],[0,1],[5,0],[0.9,1],[1,1]] 
nbrs = NearestNeighbors(n_neighbors=1, algorithm='ball_tree').fit(X)
distances, indices = nbrs.kneighbors(Y)
distances

In [ ]:
indices

In [ ]:
trial = [[0,0,0],[1,1,1]]
trial_test = [[0,0,2],[0,5,0],[5,1,0],[5,0,0]] 
trial = np.array(trial)
trial_test = np.array(trial_test)

tree = neighbors.KDTree(trial, metric = 'euclidean')
ind =  tree.query_radius(trial_test, r =5)
# print("distance to closest neighbour is: ", dist) # distance between entry of trial_test and closest neighbour in trial
print("index of closest neighbour", ind) # indices from trial that are within radius r of entry in trial_test

### back to bump

In [ ]:
hd = fits.open('galdepth10.fits', format='ascii')
new_depths = Table(hd[1].data) # depth_norm : 0 at the edge of a void to 1 at the deepest.
gal_depth = new_depths['GAL_ID'][(new_depths['DEPTH_NORM'] != -1)]
norm_depth = new_depths['DEPTH_NORM'][(new_depths['DEPTH_NORM'] != -1)]

In [ ]:
plt.hist(new_depths[data['redshift']< 0.1026]['DEPTH_NORM'][void_vf_excess],bins=np.linspace(0,1, 40), density=True, alpha = 0.5,color="red", label="void VF out of void V2 sample")
plt.hist(new_depths[data['redshift']< 0.1026]['DEPTH_NORM'][wall_vf_excess],bins=np.linspace(0,1, 40), density=True, alpha = 0.5,color="black", label="wall VF out of void V2 sample")
plt.legend();

#     ax.hist(bins_wall[:-1], bins=bins_wall, weights=counts_wall, label = 'Wall' , color=color_wall, histtype='step',linestyle=('dashed'))

In [ ]:
plt.figure(figsize=(10,5))
plt.hist(new_depths[data['redshift']< 0.1026]['DEPTH_NORM'], bins=np.linspace(0,1, 40), density=True, label="all V^2 void galaxies", color='blue', alpha=0.8, histtype='step')
plt.hist(new_depths[data['redshift']< 0.1026]['DEPTH_NORM'][void_v2_excess], bins=np.linspace(0,1, 40), density=True, alpha = 0.5,color="red", label="V^2 void galaxies -18.2<M_abs<-16.8", histtype='step')
# plt.ylim(0,100000)

plt.xlabel("normalized distance from edge")
plt.ylabel("normalized")
plt.legend(fontsize=14)
plt.xlim(-0.05,1);

The $x$-axis is the closest distance from the position of the galaxy to the edge of the void which is defined as all of the Voronoi cell edges that make up the boundary between a cell inside the void and a cell outside of it.  Thus, 0 is at the edge and 1 is at the center.  The distance to the edge of each galaxy is normalized by the largest distance to the edge in the host void.

In [ ]:
plt.figure(figsize=(15,8))

# plt.hist(new_depths[data['redshift']< 0.1026]['DEPTH_NORM'], bins=np.linspace(0,1, 40), density=True, label="all V^2 void galaxies", color='cyan', alpha=0.8, histtype='step')
# plt.hist(new_depths[data['redshift']< 0.1026]['DEPTH_NORM'][void_v2_excess], bins=np.linspace(0,1, 40), density=True, alpha = 0.5,color="orange", label="V^2 void galaxies -18.2<M_abs<-16.8", histtype='step')

plt.hist(new_depths[data['redshift']< 0.1026]['DEPTH_NORM'][all_survey['flag_voidfinder'] == 1],bins=np.linspace(0,1, 40), density=True, alpha = 0.5,color="green", label="void VF out of void V2 sample",histtype='step')
plt.hist(new_depths[data['redshift']< 0.1026]['DEPTH_NORM'][all_survey['flag_voidfinder'] == 0],bins=np.linspace(0,1, 40), density=True, alpha = 0.5,color="blue", label="wall VF out of void V2 sample",histtype='step')

plt.hist(new_depths[data['redshift']< 0.1026]['DEPTH_NORM'][void_vf_excess],bins=np.linspace(0,1, 40), density=True, alpha = 0.5,color="red", label="void VF out of void V2 sample -18.2<M_abs<-16.8",histtype='step')
plt.hist(new_depths[data['redshift']< 0.1026]['DEPTH_NORM'][wall_vf_excess],bins=np.linspace(0,1, 40), density=True, alpha = 0.5,color="black", label="wall VF out of void V2 sample -18.2<M_abs<-16.8",histtype='step')
# plt.ylim(0,100000)

plt.xlabel("normalized distance from edge")
plt.ylabel("normalized")
plt.legend(fontsize=14)
plt.xlim(-0.05,1);

In [ ]:
plt.figure(figsize=(15,8))

# plt.hist(new_depths[data['redshift']< 0.1026]['DEPTH_NORM'], bins=np.linspace(0,1, 40), density=True, label="all V^2 void galaxies", color='cyan', alpha=0.8, histtype='step')
plt.hist(new_depths[data['redshift']< 0.1026]['DEPTH_NORM'][void_v2_excess], bins=np.linspace(0,1, 40), density=True, alpha = 0.5,color="orange", label="V^2 void galaxies -18.2<M_abs<-16.8", histtype='step')

plt.hist(new_depths[data['redshift']< 0.1026]['DEPTH_NORM'][all_survey['flag_voidfinder'] == 1],bins=np.linspace(0,1, 40), density=True, alpha = 0.5,color="green", label="void VF out of void V2 sample",histtype='step')
plt.hist(new_depths[data['redshift']< 0.1026]['DEPTH_NORM'][all_survey['flag_voidfinder'] == 0],bins=np.linspace(0,1, 40), density=True, alpha = 0.5,color="blue", label="wall VF out of void V2 sample",histtype='step')

plt.hist(new_depths[data['redshift']< 0.1026]['DEPTH_NORM'][void_vf_excess],bins=np.linspace(0,1, 40), density=True, alpha = 0.5,color="red", label="void VF out of void V2 sample -18.2<M_abs<-16.8",histtype='step')
# plt.hist(new_depths[data['redshift']< 0.1026]['DEPTH_NORM'][wall_vf_excess],bins=np.linspace(0,1, 40), density=True, alpha = 0.5,color="black", label="wall VF out of void V2 sample -18.2<M_abs<-16.8",histtype='step')
# plt.ylim(0,100000)

plt.xlabel("normalized distance from edge")
plt.ylabel("normalized")
plt.legend(fontsize=14)
plt.xlim(-0.05,1);

In [ ]:
# plt.figure(figsize=(15,8))

# plt.hist(new_depths[data['redshift']< 0.1026]['DEPTH_NORM'], bins=np.linspace(0,1, 40), density=True, label="all V^2 void galaxies", color='cyan', alpha=0.8, histtype='step')
# plt.hist(new_depths[data['redshift']< 0.1026]['DEPTH_NORM'][void_v2_excess], bins=np.linspace(0,1, 40), density=True, alpha = 0.5,color="orange", label="V^2 void galaxies -18.2<M_abs<-16.8", histtype='step')

# # plt.hist(new_depths[data['redshift']< 0.1026]['DEPTH_NORM'][all_survey['flag_voidfinder'] == 1],bins=np.linspace(0,1, 40), density=True, alpha = 0.5,color="green", label="void VF out of void V2 sample",histtype='step')
# # plt.hist(new_depths[data['redshift']< 0.1026]['DEPTH_NORM'][all_survey['flag_voidfinder'] == 0],bins=np.linspace(0,1, 40), density=True, alpha = 0.5,color="blue", label="wall VF out of void V2 sample",histtype='step')

# plt.hist(new_depths[data['redshift']< 0.1026]['DEPTH_NORM'][void_vf_excess],bins=np.linspace(0,1, 40), density=True, alpha = 0.1,color="red", label="void VF out of void V2 sample -18.2<M_abs<-16.8")
# plt.hist(new_depths[data['redshift']< 0.1026]['DEPTH_NORM'][wall_vf_excess],bins=np.linspace(0,1, 40), density=True, alpha = 0.2,color="black", label="wall VF out of void V2 sample -18.2<M_abs<-16.8")
# # plt.ylim(0,100000)

# plt.xlabel("normalized distance from edge")
# plt.ylabel("normalized")
# plt.legend(fontsize=14)
# plt.xlim(-0.05,1);

Both $V^2$ voids are in the middle. void VF tend away from the edge and void wall tend towards the edge as expected.

In [ ]:
for i in [-13,-14,-15,-16,-17,-18,-19,-20,-21,-22]:
    a = i#np.random.uniform(-22+(18.2-16.8),-16+(18.2-16.8))
    b = a-(18.2-16.8)
    print(a,b)

    wall_vf_rand = np.logical_and(np.logical_and(all_survey['rabsmag']>b,all_survey['rabsmag']<a), all_survey['flag_voidfinder'] == 0)
    void_vf_rand = np.logical_and(np.logical_and(all_survey['rabsmag']>b,all_survey['rabsmag']<a), all_survey['flag_voidfinder'] == 1)

    wall_v2_rand = np.logical_and(np.logical_and(all_survey['rabsmag']>b,all_survey['rabsmag']<a), all_survey['flag_vsquared'] == 0)
    void_v2_rand = np.logical_and(np.logical_and(all_survey['rabsmag']>b,all_survey['rabsmag']<a), all_survey['flag_vsquared'] == 1)

    plt.figure(figsize=(15,8))

    # plt.hist(new_depths[data['redshift']< 0.1026]['DEPTH_NORM'], bins=np.linspace(0,1, 40), density=True, label="all V^2 void galaxies", color='cyan', alpha=0.8, histtype='step')
    # plt.hist(new_depths[data['redshift']< 0.1026]['DEPTH_NORM'][void_v2_excess], bins=np.linspace(0,1, 40), density=True, alpha = 0.5,color="orange", label="V^2 void galaxies -18.2<M_abs<-16.8", histtype='step')

    plt.hist(new_depths[data['redshift']< 0.1026]['DEPTH_NORM'][all_survey['flag_voidfinder'] == 1],bins=np.linspace(0,1, 40), density=True, alpha = 0.5,color="green", label="void VF out of void V2 sample",histtype='step')
    plt.hist(new_depths[data['redshift']< 0.1026]['DEPTH_NORM'][void_vf_rand],bins=np.linspace(0,1, 40), density=True, alpha = 0.5,color="red", label="void VF out of void V2 sample random",histtype='step')

    plt.hist(new_depths[data['redshift']< 0.1026]['DEPTH_NORM'][all_survey['flag_voidfinder'] == 0],bins=np.linspace(0,1, 40), density=True, alpha = 0.5,color="blue", label="wall VF out of void V2 sample",histtype='step')

    plt.hist(new_depths[data['redshift']< 0.1026]['DEPTH_NORM'][wall_vf_rand],bins=np.linspace(0,1, 40), density=True, alpha = 0.5,color="black", label="wall VF out of void V2 sample random",histtype='step')
    # plt.ylim(0,100000)

    plt.xlabel("normalized distance from edge")
    plt.ylabel("normalized")
    plt.legend(fontsize=14)
    plt.xlim(-0.05,1);

In [ ]:
ind_v2_void = new_depths['GAL_ID'][data['flag_vsquared'] == 1] # void vf

In [ ]:
# new_depths[data['redshift']< 0.1026]['DEPTH_NORM'][all_survey['flag_voidfinder'] == 1] # void vf

In [ ]:
ind_vf_wall = new_depths['GAL_ID'][data['flag_voidfinder'] == 0] # void vf
ind_vf_void = new_depths['GAL_ID'][data['flag_voidfinder'] == 1] # void vf

In [ ]:
plt.figure(figsize=(12,7))

plt.scatter(data['rabsmag'][ind_vf_wall],new_depths['DEPTH_NORM'][data['flag_voidfinder'] == 0],alpha=0.1, s=1) # void vf
plt.ylim(0,1)
plt.xlim(-12,-24)
plt.xlabel("absolute magnitude", fontsize=14)
plt.title("WALL VF sample with depths")
plt.ylabel("distance from edge", fontsize=14);
# the deeper objects are brighter

In [ ]:
plt.figure(figsize=(12,7))
plt.scatter(data['rabsmag'][ind_vf_void],new_depths['DEPTH_NORM'][data['flag_voidfinder'] == 1],alpha=0.1, s=1) # void vf
plt.ylim(0,1)
plt.xlim(-12,-24)
plt.xlabel("absolute magnitude", fontsize=14)
plt.title("VOID VF sample with depths")
plt.ylabel("distance from edge", fontsize=14);
# the deeper objects are brighter

In [ ]:
plt.figure(figsize=(12,7))

plt.scatter(data['rabsmag'][ind],new_depths['DEPTH_NORM'][data['flag_vsquared'] == 1],alpha=0.1, s=1) # void vf
plt.ylim(0,1)
plt.xlim(-12,-24)
plt.xlabel("absolute magnitude", fontsize=14)
plt.title("Void V2 sample")
plt.ylabel("distance from edge", fontsize=14)
# the deeper objects are brighter # voidfinder depths # slides

In [ ]:
plt.figure(tight_layout=True)

plt.scatter(new_depths['DEPTH_NORM'][data['flag_voidfinder'] == 1],data['rabsmag'][ind]) # void vf
plt.xlim(0,1)
plt.ylabel("absolute magnitude")
plt.title("VOID VF sample with depths")
plt.xlabel("distance from edge");

# for i in [-13,-14,-15,-16,-17,-18,-19,-20,-21,-22]:
#     a = i#np.random.uniform(-22+(18.2-16.8),-16+(18.2-16.8))
#     b = a-(18.2-16.8)
#     print(a,b)

#     wall_vf_rand = np.logical_and(np.logical_and(all_survey['rabsmag']>b,all_survey['rabsmag']<a), all_survey['flag_voidfinder'] == 0)
#     void_vf_rand = np.logical_and(np.logical_and(all_survey['rabsmag']>b,all_survey['rabsmag']<a), all_survey['flag_voidfinder'] == 1)

#     wall_v2_rand = np.logical_and(np.logical_and(all_survey['rabsmag']>b,all_survey['rabsmag']<a), all_survey['flag_vsquared'] == 0)
#     void_v2_rand = np.logical_and(np.logical_and(all_survey['rabsmag']>b,all_survey['rabsmag']<a), all_survey['flag_vsquared'] == 1)

#     plt.figure(figsize=(15,8))

#     # plt.hist(new_depths[data['redshift']< 0.1026]['DEPTH_NORM'], bins=np.linspace(0,1, 40), density=True, label="all V^2 void galaxies", color='cyan', alpha=0.8, histtype='step')
#     # plt.hist(new_depths[data['redshift']< 0.1026]['DEPTH_NORM'][void_v2_excess], bins=np.linspace(0,1, 40), density=True, alpha = 0.5,color="orange", label="V^2 void galaxies -18.2<M_abs<-16.8", histtype='step')

#     plt.hist(new_depths[data['redshift']< 0.1026]['DEPTH_NORM'][all_survey['flag_voidfinder'] == 1],bins=np.linspace(0,1, 40), density=True, alpha = 0.5,color="green", label="void VF out of void V2 sample",histtype='step')
#     plt.hist(new_depths[data['redshift']< 0.1026]['DEPTH_NORM'][void_vf_rand],bins=np.linspace(0,1, 40), density=True, alpha = 0.5,color="red", label="void VF out of void V2 sample random",histtype='step')

#     plt.hist(new_depths[data['redshift']< 0.1026]['DEPTH_NORM'][all_survey['flag_voidfinder'] == 0],bins=np.linspace(0,1, 40), density=True, alpha = 0.5,color="blue", label="wall VF out of void V2 sample",histtype='step')

#     plt.hist(new_depths[data['redshift']< 0.1026]['DEPTH_NORM'][wall_vf_rand],bins=np.linspace(0,1, 40), density=True, alpha = 0.5,color="black", label="wall VF out of void V2 sample random",histtype='step')
#     # plt.ylim(0,100000)

#     plt.xlabel("normalized distance from edge")
#     plt.ylabel("normalized")
#     plt.legend(fontsize=14)
#     plt.xlim(-0.05,1);

In [ ]:
a,b

In [ ]:
np.array(np.histogram(new_depths[data['redshift']< 0.1026]['DEPTH_NORM'][all_survey['flag_voidfinder'] == 1],
                      bins=np.linspace(0,1, 40))[0])

In [ ]:
a,b

In [ ]:
plt.figure(tight_layout=True)

plt.hist(new_depths[data['redshift']< 0.1026]['DEPTH_NORM'][void_v2_excess], 
         bins=30);

In [ ]:
# plt.hist(new_depths['DEPTH_NORM'],bins=np.linspace(0,1,100), density=True, label="all")

# plt.hist(new_depths['DEPTH_NORM'][np.intersect1d(wall_v2,bump_mstar_index)],bins=np.linspace(0,1,100), density=True, alpha=0.3)
# plt.legend();

In [ ]:
# len(np.intersect1d(void_v2,bump_mstar_index))

In [ ]:
plt.figure(tight_layout=True)

plt.hist(new_depths['DEPTH_NORM'],bins=np.linspace(0,1,100), density=True, label="all")

plt.hist(new_depths['DEPTH_NORM'][np.intersect1d(wall_v2,bump_mstar_index)],bins=np.linspace(0,1,100), density=True, alpha=0.3)
plt.legend();

In [ ]:

# gal_sorted = gal_depth[np.flip(np.argsort(norm_depth))] # galaxies sorted by largest depth to smallest
# # norm_depth[np.argwhere(gal_depth == X)]  # to find norm_depth associated with galaxy X
# #check that the galaxies are sorted in ascending order of the normalized depths
# gal_sorted[500] == gal_depth[np.argwhere(norm_depth==np.flip(np.sort(norm_depth))[500])]


# num_slices = 50
# last_slice_count =len(gal_sorted[:(np.int(len(gal_sorted)/num_slices))*num_slices])

# ind_gal_depth_voidv2_new = []
# temp=[]
# # for i in binning_log_e:
# #     temp.append(new_depths['GAL_ID'][(new_depths['DEPTH_NORM'] != -1) &(new_depths['DEPTH_NORM']>i)])
# #     ind_gal_depth_voidv2_new.append(temp)
# #     temp=[]

# for i in range(num_slices):
#     temp.append(gal_sorted[:(np.int(len(gal_sorted)/num_slices))*(i+1)])
#     ind_gal_depth_voidv2_new.append(temp)
#     temp=[]  
# for i in range(len(gal_sorted)-last_slice_count):
#     ind_gal_depth_voidv2_new[-1][0] = np.append(gal_sorted[last_slice_count:len(gal_sorted)][i],ind_gal_depth_voidv2_new[-1][0])
    
# ind_gal_depth_voidv2_new = np.array(ind_gal_depth_voidv2_new).reshape((num_slices,))

# #check
# len(ind_gal_depth_voidv2_new[-1]) == len(gal_sorted)

# # np.save('ind_gal_depth_voidv2_50_log_e', ind_gal_depth_voidv2_new)
# np.save('ind_gal_depth_voidv2_equal_slices', ind_gal_depth_voidv2_new)

# # sorted_gal_in=[]
# # for i in range(len(norm_depth)):
# #     sorted_gal_in.append(np.argwhere(norm_depth == np.flip(np.sort(norm_depth))[i]))
# # sorted_gal=[]
# # for i in range(len(gal_depth)):
# #     sorted_gal.append(gal_depth[np.array(sorted_gal_in)[i].flatten().astype('int')])
# # sorted_gal_int=[]
# # for i in range(len(sorted_gal)):
# #     for k in range(len(sorted_gal[i])):
# #         sorted_gal_int.append(sorted_gal[i][k])  
# # sorted_gal_int= np.array(sorted_gal_int)

# # binning = np.abs(np.log(np.linspace(1,0.01,80)))
# # binning = binning[binning<1]
# # binning_log_e= np.append(binning,1)
# # binning_log_e = np.flip(binning_log_e)[1:]
# # binning_log_e

# # ind_gal_depth_voidv2_new = []
# # temp=[]
# # for i in binning_log_e:
# #     temp.append(new_depths['GAL_ID'][(new_depths['DEPTH_NORM'] != -1) &(new_depths['DEPTH_NORM']>i)])
# #     ind_gal_depth_voidv2_new.append(temp)
# #     temp=[]

In [ ]:
# void_depth = new_depths[void_v2]['DEPTH_NORM']
# void_depth_ind = new_depths[np.argwhere(new_depths['DEPTH_NORM']==new_depths[void_v2]['DEPTH_NORM'])]

In [ ]:
# void_depth_ind=[]
# for i in range(len(void_depth)):
#     void_depth_ind.append()

In [ ]:
# void_depth_bump=np.intersect1d(bump_mstar_index,void_depth_ind)

In [ ]:
# fig = plt.figure(figsize=(15,10))
# plt.hist(new_depths[void_bump]['DEPTH_NORM'], np.linspace(-2,1,300), label="bump")
# plt.hist(random.sample(new_depths[np.argwhere(new_depths[void_v2]['DEPTH_NORM']!= -1)]['DEPTH_NORM'],len(void_bump)), np.linspace(-2,1,300), alpha=0.5, label="random")
# plt.legend();

In [ ]:
Mstar_nan = remove_nan(Mstar)
Mstar_nan_NSA = remove_nan(Mstar_NSA)

In [ ]:
len(Mstar_nan_NSA),len(Mstar_nan)

In [ ]:
Mstar_nan=np.nan_to_num(Mstar, copy=True, nan=0.0, posinf=0, neginf=0)

In [ ]:
bump_1= np.argwhere(Mstar_nan<9.5)
bump_2= np.argwhere(Mstar_nan>8.7)

In [ ]:
test=np.intersect1d(np.argwhere(Mstar_nan>9.7),np.argwhere(Mstar_nan<10.5))

In [ ]:
test_=random.sample(list(test),35526)
len(test_)

In [ ]:
bump_mstar_index = np.intersect1d(bump_1,bump_2)

In [ ]:
len(bump_mstar_index) # the indices of galaxies whose mass is between 9.5 and 8.7

In [ ]:
len(np.intersect1d(bump_mstar_index, void_v2)), len(np.intersect1d(bump_mstar_index, wall_v2))

In [ ]:
fig = plt.figure(figsize=(15,10), tight_layout=True)

# plt.hist(Mstar_nan[np.array(data['flag_voidfinder'])==9], bins=np.linspace(5.9,12, 135), label="outside",histtype='step', color="black", density=True)
plt.hist(Mstar_nan[np.array(data['flag_voidfinder'])==0], bins=np.linspace(5.9,12, 135), label="wall",histtype='step', color="blue",density=True)
# plt.hist(Mstar_nan[np.array(data['flag_voidfinder'])==2], bins=np.linspace(5.9,12, 135), label="edge",histtype='step', color="purple",density=True)
plt.hist(Mstar_nan[np.array(data['flag_voidfinder'])==1], bins=np.linspace(5.9,12, 135), color="red",label="void", histtype='step',density=True)

# plt.hist(Mstar_nan[np.array(data['flag_vsquared'])==9], bins=np.linspace(5.9,12, 135), label="outside v2",linestyle=('dashed'),histtype='step', color="black",density=True)
plt.hist(Mstar_nan[np.array(data['flag_vsquared'])==0], bins=np.linspace(5.9,12, 135), label="wall v2",linestyle=('dashed'),histtype='step', color="blue",density=True)
# plt.hist(Mstar_nan[np.array(data['flag_vsquared'])==2], bins=np.linspace(5.9,12, 135), label="edge v2",linestyle=('dashed'),histtype='step', color="purple",density=True)
plt.hist(Mstar_nan[np.array(data['flag_vsquared'])==1], bins=np.linspace(5.9,12, 135), color="red",linestyle=('dashed'),label="void v2", histtype='step',density=True)

plt.hist(Mstar_nan[data['redshift']< 0.1026], bins=np.linspace(5.9,12, 135), color = 'gray',alpha = 0.2,density=True)
plt.legend();

In [ ]:
#wall=0 , void = 1, edge= 2, out = 9
print("wall")
print(np.sum(np.array(data['flag_voidfinder'][bump_mstar_index])==0)- np.sum(np.array(data['flag_vsquared'][bump_mstar_index])==0))
print("void")
print(np.sum(np.array(data['flag_voidfinder'][bump_mstar_index])==1)- np.sum(np.array(data['flag_vsquared'][bump_mstar_index])==1))
print("edge")
print(np.sum(np.array(data['flag_voidfinder'][bump_mstar_index])==2)- np.sum(np.array(data['flag_vsquared'][bump_mstar_index])==2))
print("out")
print(np.sum(np.array(data['flag_voidfinder'][bump_mstar_index])==9)- np.sum(np.array(data['flag_vsquared'][bump_mstar_index])==9))

In [ ]:
np.sum(np.array(data['flag_voidfinder'][bump_mstar_index])==1), 
np.sum(np.array(data['flag_vsquared'][bump_mstar_index])==1)

In [ ]:
np.sum(np.array(data['flag_voidfinder'][bump_mstar_index])==1), 
np.sum(np.array(data['flag_vsquared'][bump_mstar_index])==1)

In [ ]:
fig = plt.figure(figsize=(10,5), tight_layout=True)

plt.hist(random.sample(list(data['flag_voidfinder']),35526), label="voidfinder")
plt.hist(random.sample(list(data['flag_vsquared']),35526), color = "orange", alpha=0.3, label="vsquared")
plt.legend();

In [ ]:
#wall=0 , void = 1, edge= 2, out = 9
fig = plt.figure(figsize=(10,5), tight_layout=True)

plt.hist(np.array(data['flag_voidfinder'][test_]), label="voidfinder")
plt.hist(np.array(data['flag_vsquared'][test_]), color = "orange", alpha=0.3, label="vsquared")
plt.legend();

There are much more void galaxies in $V^2$ than VoidFinder in this region.

In [ ]:
plt.hist(np.array(data['flag_voidfinder']), label="voidfinder")
plt.hist(np.array(data['flag_vsquared']), color = "orange", alpha=0.3, label="vsquared")
plt.legend();

In [ ]:
#wall=0 , void = 1, edge= 2, out = 9
fig = plt.figure(figsize=(10,5))
plt.hist(np.array(data['flag_voidfinder'][bump_mstar_index]), label="voidfinder")
plt.hist(np.array(data['flag_vsquared'][bump_mstar_index]), color = "orange", alpha=0.3, label="vsquared")
plt.legend();

There are much more void galaxies in $V^2$ than VoidFinder in this region.

In [ ]:
fig = plt.figure(figsize=(10,5))
plt.hist(np.array(data['flag_voidfinder'][35526+35526:35526+35526+35526]), label="voidfinder")
plt.hist(np.array(data['flag_vsquared'][35526+35526:35526+35526+35526]), alpha=0.3, label="vsquared")
plt.legend();

In [ ]:
fig = plt.figure(figsize=(15,10))
plt.hist(Mstar_nan[wall_v2], bins=np.linspace(5.9,12, 135), label="wall v2")
plt.hist(Mstar_nan[void_v2], bins=np.linspace(5.9,12, 135), alpha=0.3, label="void v2")
plt.hist(Mstar_nan[data['redshift']< 0.1026], bins=np.linspace(5.9,12, 135), color = 'gray',alpha = 0.2)

plt.legend();

In [ ]:
fig = plt.figure(figsize=(15,10))
plt.hist(Mstar_nan[wall_vf], bins=np.linspace(5.9,12, 135), label="wall vf")
plt.hist(Mstar_nan[void_vf], bins=np.linspace(5.9,12, 135), alpha=0.3, label="void vf")
plt.hist(Mstar_nan[data['redshift']< 0.1026], bins=np.linspace(5.9,12, 135), color = 'gray',alpha = 0.2)

plt.legend();

In [ ]:
char(Mstar_nan,np.linspace(5.9,12, 135),r'Star Mass - Voidfinder',r'$\log(M_*/M_\odot)$',x=6,y=00.025)